# Shamir Secret Sharing (SS)

This is demo for our implementation of Shamir's secret sharing. We choose the Field $\mathbb{Z}_p$ where $p > 2^\lambda$ is a prime ($\lambda$ is the key size)

In [1]:
from ftsa.protocols.buildingblocks.ShamirSS import SSS
import random

threshold = 70
nclients  = 100
SS = SSS(1024)

number1 = random.randint(0,2**1024-1)
number2 = random.randint(0,2**1024-1)

allshares1 = SS.share(threshold, nclients, number1)
allshares2 = SS.share(threshold, nclients, number2)

random.seed(1)
shares1 = random.sample(allshares1, threshold)
random.seed(1)
shares2 = random.sample(allshares2, threshold)

shares = [x+y for x,y in zip(shares1, shares2)]

lagcoef = SS.lagrange(shares1)
reconstucted1 =  SS.recon(shares1, lagcoef)
reconstucted2 =  SS.recon(shares2, lagcoef)
reconstucted =  SS.recon(shares, lagcoef)

# check if the result is correct
print("Verify1: ", number1 == reconstucted1)
print("Verify2: ", number2 == reconstucted2)
print("Verify Sum: ", (number2 + number1) == reconstucted)


Verify1:  True
Verify2:  True
Verify Sum:  True


# Shamir Secret Sharing ove the Integers (ISS)

This is demo for our implementation of Shamir's secret sharing over the integers. We choose the security parameter sigma to $\sigma = 128$ bits

In [2]:
from ftsa.protocols.buildingblocks.IntegerSS import ISSS
import random
from math import factorial

sigma = 128
threshold = 70
nclients  = 100
delta = factorial(nclients)

SS = ISSS(1024, sigma)

number1 = random.randint(-2**1024+1,2**1024-1)
number2 = random.randint(-2**1024+1,2**1024-1)

allshares1 = SS.Share(number1,threshold, range(1,nclients+1))
allshares2 = SS.Share(number2,threshold, range(1,nclients+1))

random.seed(1)
shares1 = random.sample(allshares1, threshold)
random.seed(1)
shares2 = random.sample(allshares2, threshold)

shares = [x+y for x,y in zip(shares1, shares2)]

reconstucted1 =  SS.Recon(shares1, threshold, delta)
reconstucted2 =  SS.Recon(shares2, threshold, delta)
reconstucted =  SS.Recon(shares, threshold, delta)

# check if the result is correct
print("Verify1: ", number1 == reconstucted1)
print("Verify2: ", number2 == reconstucted2)
print("Verify Sum: ", (number2 + number1) == reconstucted)


Verify1:  True
Verify2:  True
Verify Sum:  True


# Deffie-Hellman Key Aggreement (KA)
It uses ECDH p256 curve. It then computes with SHA256 over the ECDH agreed key to generate a secret key for AES (256 bit)

In [3]:
from ftsa.protocols.buildingblocks.KeyAggreement import KAS

sharedkeysize = 1024

KA1 = KAS()
KA2 = KAS()

KA1.generate()
KA2.generate()

# check if the shared secrets match
print("Verify:", KA1.agree(KA2.pk, sharedkeysize) == KA2.agree(KA1.pk, sharedkeysize))

# print shared key
sharedkey = KA1.agree(KA2.pk, sharedkeysize)
print("key: " + str(sharedkey))

Verify: True
key: 173778870878702737440328737816104008308541745960814777371653569995596744011764896828048701077620952755594337846137305376840882561284829959117550302614941951234582038266748141938715532799325596795428057932214668285112764894056663140372442458797997954711502166545331124274780960560545852115034375979649017272750


In [4]:
# test serializing secret keys
sk1_bytes = KA1.get_sk_bytes()
KA = KAS()
KA.generate_from_bytes(sk1_bytes)
print("Verify:", KA.sk == KA1.sk and  KA.pk == KA1.pk)

Verify: True


# AES-GCM Authenticated Encryption (AE)

In [5]:
import ftsa.protocols.buildingblocks.AESGCM128 as aes

message=b"TOP SECRET"

key = aes.EncryptionKey(sharedkey)
e = key.encrypt(message)
m = key.decrypt(e)
print("Verify:", message == m)

Verify: True


# Vector Encoding Scheme (VE)
This is a vector to vector encoding. It batches several elements of the vectors into a single integer. The batch size depends on the size of that integer (i.e. this means the plaintext size of an encryption scheme). It also depends on the number of safety bits need to be preserved for each vector element (this is to support addition operation without reaching an overflow)

In [6]:
from ftsa.protocols.buildingblocks.VectorEncoding import VES

keysize = 1024
nclients = 100
valuesize = 16
vectorsize = 100

VE = VES(keysize, nclients, valuesize, vectorsize)
vector = list(range(1,vectorsize+1))

# encode the vector 
E = VE.encode(V=vector)
print("encoded decreased the vector size from {} to {}".format(len(vector),len(E)))
# decode the vector 
V = VE.decode(E=E)
print("Verify:", V == vector)

encoded decreased the vector size from 100 to 3
Verify: True


# Pseudo Randomg Generator (PRG)
## We use AES-CTR to generate a pseudo random vector from a seed

The demo first generates a random seed. The seed is used as the AES key to encrypt a '0x00'-repeated message of large size. The encryption result is transformed into a vector of integers. The output vector size is controlled by the size of the input message.

In [7]:
from ftsa.protocols.buildingblocks.PRG import PRG
import random

securitybits = 128 #bits
elementsize = 16 #bits
vectorsize = 2**20 #elements

# setup the PRG 
prg = PRG(vectorsize, elementsize)

In [8]:
# get a random seed of 16 bytes long
b = random.SystemRandom().getrandbits(securitybits).to_bytes(securitybits // 8,"big")

# generate a random vector from that seed 
B = prg.eval(b)

print("Generated a pseudo random vector of length {} ({} bits elements) from {} bits random value".format(len(B), elementsize, securitybits))
B

Generated a pseudo random vector of length 1048576 (16 bits elements) from 128 bits random value


[47050,
 6965,
 16156,
 54971,
 59642,
 44524,
 13882,
 15883,
 20515,
 17374,
 60758,
 42267,
 10031,
 60702,
 53218,
 9227,
 41525,
 38164,
 21831,
 61619,
 23233,
 50073,
 12065,
 47033,
 8812,
 45585,
 1204,
 8139,
 60222,
 19802,
 51359,
 28814,
 33445,
 50682,
 32650,
 34091,
 18155,
 43326,
 45077,
 43783,
 45627,
 58091,
 18757,
 40894,
 8780,
 38640,
 38152,
 48609,
 9234,
 40443,
 48209,
 36513,
 39730,
 47512,
 32145,
 58542,
 39029,
 26369,
 6764,
 3394,
 20868,
 46436,
 20663,
 56427,
 40961,
 24852,
 61092,
 60456,
 34397,
 29360,
 7112,
 50153,
 20163,
 27137,
 56852,
 58260,
 53697,
 26287,
 14286,
 59014,
 20147,
 10901,
 36363,
 59555,
 19932,
 60085,
 18738,
 25097,
 20352,
 55004,
 11315,
 11897,
 8816,
 42615,
 50587,
 44674,
 4511,
 57523,
 39430,
 9193,
 25490,
 31328,
 44859,
 37669,
 44228,
 47797,
 9818,
 16750,
 61128,
 58050,
 40027,
 40639,
 40132,
 64168,
 9981,
 44726,
 65186,
 30664,
 37872,
 50667,
 31948,
 54542,
 56377,
 50445,
 34474,
 33649,
 45763,